In [25]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import sys
import timeit
import heapq 

In [26]:
resultA = []
resultB = []

In [27]:
class Graph():
    
    def __init__(self, vertices):
        self.V = vertices
        self.matrixgraph = []
        self.listgraph = {}
    
    def generateNew(self):
        self.matrixgraph = np.random.randint(0,10, (self.V, self.V)) #Generate random adj matrix + set limit of weights
        
        for i in range(self.V):                                     #Set diagonal of adj matrix to 0, this is to eliminate loops
            for j in range(self.V):
                if i == j:
                    self.matrixgraph[i][j] = 0
                    
        for i in range(2, self.V):               # Assumes that node 0 is our source node, eliminates all other edges except
            self.matrixgraph[0][i] = 0           # the edge from node 0 -> node 1.
        for i in range(1, self.V):
            self.matrixgraph[i][0] = 0
        
        if self.matrixgraph[0][1] == 0:          # Incase of bad RNG, form connection from source to node 1
            self.matrixgraph[0][1] = 1
            
        self.convertToAdj()                                         #Convert from Adj matrix to Adj list
                    
    def getMatrix(self):                                            #Prints the adj matrix
        print("Adjacency Matrix for the graph,")
        print(g.matrixgraph)
    
    def getList(self):                                              #Prints the adj list
        print("Adjacency List for the graph,")
        for nodes in self.listgraph:
            print(nodes, end = "")
            for edges in self.listgraph[nodes]:
                print(" -> ", edges[0], "Weight:", edges[1], end = "")
            print()
            
    def visualize(self):                                            #Visualize the graph
        G = nx.from_numpy_matrix(np.matrix(g.matrixgraph), create_using=nx.DiGraph)
        layout = nx.spring_layout(G)
        nx.draw(G, layout, with_labels=True)
        labels = nx.get_edge_attributes(G, "weight")
        nx.draw_networkx_edge_labels(G, pos=layout, edge_labels=labels)
        plt.show()
        
    def convertToAdj(self):
        for i in range(self.V):                                     #Creation of adj list using adj matrix
            self.listgraph[i] = []
        for i in range(self.V):
            for j in range(self.V):
                if self.matrixgraph[i][j] != 0:
                    temp = [j, self.matrixgraph[i][j]]
                    self.listgraph[i].append(temp)

In [28]:
g = Graph(1000)
g.generateNew()
g.getMatrix()
print()
# g.getList()
# print()
# g.visualize()

Adjacency Matrix for the graph,
[[0 8 0 ... 0 0 0]
 [0 0 7 ... 5 0 1]
 [0 8 0 ... 1 7 0]
 ...
 [0 1 6 ... 0 0 5]
 [0 9 3 ... 2 0 3]
 [0 1 8 ... 6 2 0]]



Part (a) - Implementation of Dijkstra's algorithm using adjacency matrix + array for priority queue

In [29]:
class Node():
    
    def __init__(self, vertex, weight):
        self.vertex = vertex
        self.weight = weight

In [30]:
class PriorityQueue:

    def __init__(self):
        self.q = []

    def insert(self, node):
        if len(self.q) == 0:
            self.q.append(node)
        else:
            for i in range(len(self.q)):
                if node.weight < self.q[i].weight:
                    self.q.insert(i, node)
                    break
                elif node.weight == self.q[i].weight:
                    pass

                if i == len(self.q) - 1:
                    self.q.insert(i + 1, node)

    def getMinNode(self):
        return self.q[0].vertex

    def getMinWeight(self):
        return self.q[0].weight

    def pop(self):
        self.q.pop(0)

    def show(self):
        print("Current priority queue,")
        for i in range(len(self.q)):
            print(self.q[i].vertex, "- Weight", self.q[i].weight)

    def isEmpty(self):
        return len(self.q) == 0

    def removeNode(self, index):
        for i in range(len(self.q)):
            if self.q[i].vertex == index:
                self.q.pop(i)
                break

In [31]:
def DijkstraA(G, source):
    time_start = timeit.default_timer()
    
    distance = [sys.maxsize] * G.V  # Original distance for all vertices are set to infinity
    path = [[] for i in range(G.V)]  # Initialize N number of sub-lists to store the path to each node
    S = [False] * G.V  # Initialize False to all the nodes since shortest path not yet discovered

    distance[source] = 0  # Set distance of source node to be zero
    pQueue = PriorityQueue()

    for i in range(G.V):  # Insert all the vertices into priority queue
        tempNode = Node(i, distance[i])
        pQueue.insert(tempNode)

    while (not pQueue.isEmpty()):
        u = pQueue.getMinNode()
        pQueue.pop()
        S[u] = True

        for i in range(G.V):
            if G.matrixgraph[u][i] != 0:
                if S[i] != True and distance[i] > distance[u] + G.matrixgraph[u][i]:
                    pQueue.removeNode(i)
                    distance[i] = distance[u] + G.matrixgraph[u][i]                   
                    if source not in path[i]:
                        for elements in path[u]:
                            path[i].append(elements)
                    path[i].append(u)
                    pQueue.insert(Node(i, distance[i]))
    
    time_stop = timeit.default_timer()
    #END OF DIJKSTRA ALGORITHM
    
    #Print results
    print("Time elapsed for program: ", (time_stop - time_start)*1000, "milliseconds")  # Time in milliseconds
    print("Distance from source node,")
    for i in range(0, G.V):
        if distance[i] == sys.maxsize:
            print("Source to node", i, ": No such path exist")
        else:
            print("Source to node", i, ":", distance[i])
        resultA.append(distance[i])
    
    print()
    print("Path to get to each node,")
    
    for i in range(0, G.V):
        print("Path from source to node", i)
        print(path[i])
        print()

In [32]:
DijkstraA(g, 0)

Time elapsed for program:  1999.20419999998 milliseconds
Distance from source node,
Source to node 0 : 0
Source to node 1 : 8
Source to node 2 : 10
Source to node 3 : 10
Source to node 4 : 10
Source to node 5 : 10
Source to node 6 : 10
Source to node 7 : 10
Source to node 8 : 10
Source to node 9 : 10
Source to node 10 : 10
Source to node 11 : 9
Source to node 12 : 10
Source to node 13 : 10
Source to node 14 : 10
Source to node 15 : 10
Source to node 16 : 10
Source to node 17 : 10
Source to node 18 : 10
Source to node 19 : 10
Source to node 20 : 10
Source to node 21 : 10
Source to node 22 : 10
Source to node 23 : 10
Source to node 24 : 10
Source to node 25 : 10
Source to node 26 : 10
Source to node 27 : 10
Source to node 28 : 10
Source to node 29 : 10
Source to node 30 : 9
Source to node 31 : 10
Source to node 32 : 10
Source to node 33 : 10
Source to node 34 : 10
Source to node 35 : 10
Source to node 36 : 10
Source to node 37 : 10
Source to node 38 : 10
Source to node 39 : 10
Source to 

Source to node 760 : 10
Source to node 761 : 10
Source to node 762 : 10
Source to node 763 : 10
Source to node 764 : 10
Source to node 765 : 10
Source to node 766 : 10
Source to node 767 : 10
Source to node 768 : 10
Source to node 769 : 10
Source to node 770 : 10
Source to node 771 : 10
Source to node 772 : 10
Source to node 773 : 10
Source to node 774 : 10
Source to node 775 : 9
Source to node 776 : 10
Source to node 777 : 10
Source to node 778 : 10
Source to node 779 : 10
Source to node 780 : 10
Source to node 781 : 10
Source to node 782 : 9
Source to node 783 : 10
Source to node 784 : 10
Source to node 785 : 9
Source to node 786 : 10
Source to node 787 : 10
Source to node 788 : 10
Source to node 789 : 9
Source to node 790 : 10
Source to node 791 : 10
Source to node 792 : 10
Source to node 793 : 10
Source to node 794 : 10
Source to node 795 : 10
Source to node 796 : 9
Source to node 797 : 10
Source to node 798 : 9
Source to node 799 : 10
Source to node 800 : 10
Source to node 801 : 9


Path from source to node 506
[0, 1]

Path from source to node 507
[0, 1, 11, 47]

Path from source to node 508
[0, 1]

Path from source to node 509
[0, 1, 11, 30]

Path from source to node 510
[0, 1]

Path from source to node 511
[0, 1, 11]

Path from source to node 512
[0, 1, 11]

Path from source to node 513
[0, 1]

Path from source to node 514
[0, 1, 68, 117]

Path from source to node 515
[0, 1, 11, 30, 44, 48]

Path from source to node 516
[0, 1, 11, 48, 134]

Path from source to node 517
[0, 1, 11, 47]

Path from source to node 518
[0, 1, 11, 85]

Path from source to node 519
[0, 1, 11, 30, 63, 72, 87]

Path from source to node 520
[0, 1, 44]

Path from source to node 521
[0, 1, 44]

Path from source to node 522
[0, 1, 63, 68]

Path from source to node 523
[0, 1, 11, 47]

Path from source to node 524
[0, 1, 47]

Path from source to node 525
[0, 1, 47]

Path from source to node 526
[0, 1, 70, 89]

Path from source to node 527
[0, 1, 136]

Path from source to node 528
[0, 1]

Path 

Part (b) - Implementation of Dijkstra's algorithm using adjacency list + min heap as priority queue

For this implementation, we shall use the heapq module for our min heap.

In [33]:
def DijkstraB(G, source):
    time_start = timeit.default_timer()
    
    distance = [sys.maxsize] * G.V  # Original distance for all vertices are set to infinity
    path = [[] for i in range(G.V)]  # Initialize N number of sub-lists to store the path to each node
    S = [False] * G.V  # Initialize False to all the nodes since shortest path not yet discovered

    distance[source] = 0  # Set distance of source node to be zero
    
    minHeap = []
    for i in range(G.V):  # Insert all the vertices into min heap
        heapq.heappush(minHeap, [distance[i], i])
    
    while (len(minHeap) != 0):
        u = heapq.heappop(minHeap)
        S[u[1]] = True
        
        for elements in G.listgraph[u[1]]:
            if S[elements[0]] != True and distance[elements[0]] > distance[u[1]] + elements[1]:
                for z in minHeap:
                    if z[1] == elements[0]:
                        minHeap.remove(z)
                        heapq.heapify(minHeap)
                            
                distance[elements[0]] = distance[u[1]] + elements[1]
                    
                if source not in path[elements[0]]:
                    for nodes in path[u[1]]:
                        path[elements[0]].append(nodes)
                path[elements[0]].append(u[1])
                    
                heapq.heappush(minHeap, [distance[elements[0]], elements[0]])
    
    time_stop = timeit.default_timer()
    #END OF DIJKSTRA ALGORITHM
    
    #Print results
    print("Time elapsed for program: ", (time_stop - time_start)*1000, "milliseconds")  # Time in milliseconds
    print("Distance from source node,")
    for i in range(0, G.V):
        if distance[i] == sys.maxsize:
            print("Source to node", i, ": No such path exist")
        else:
            print("Source to node", i, ":", distance[i])
        resultB.append(distance[i])
    
    print()
    print("Path to get to each node,")
    
    for i in range(0, G.V):
        print("Path from source to node", i)
        print(path[i])
        print()

In [34]:
DijkstraB(g, 0)

Time elapsed for program:  2282.3397000000227 milliseconds
Distance from source node,
Source to node 0 : 0
Source to node 1 : 8
Source to node 2 : 10
Source to node 3 : 10
Source to node 4 : 10
Source to node 5 : 10
Source to node 6 : 10
Source to node 7 : 10
Source to node 8 : 10
Source to node 9 : 10
Source to node 10 : 10
Source to node 11 : 9
Source to node 12 : 10
Source to node 13 : 10
Source to node 14 : 10
Source to node 15 : 10
Source to node 16 : 10
Source to node 17 : 10
Source to node 18 : 10
Source to node 19 : 10
Source to node 20 : 10
Source to node 21 : 10
Source to node 22 : 10
Source to node 23 : 10
Source to node 24 : 10
Source to node 25 : 10
Source to node 26 : 10
Source to node 27 : 10
Source to node 28 : 10
Source to node 29 : 10
Source to node 30 : 9
Source to node 31 : 10
Source to node 32 : 10
Source to node 33 : 10
Source to node 34 : 10
Source to node 35 : 10
Source to node 36 : 10
Source to node 37 : 10
Source to node 38 : 10
Source to node 39 : 10
Source t

Source to node 759 : 10
Source to node 760 : 10
Source to node 761 : 10
Source to node 762 : 10
Source to node 763 : 10
Source to node 764 : 10
Source to node 765 : 10
Source to node 766 : 10
Source to node 767 : 10
Source to node 768 : 10
Source to node 769 : 10
Source to node 770 : 10
Source to node 771 : 10
Source to node 772 : 10
Source to node 773 : 10
Source to node 774 : 10
Source to node 775 : 9
Source to node 776 : 10
Source to node 777 : 10
Source to node 778 : 10
Source to node 779 : 10
Source to node 780 : 10
Source to node 781 : 10
Source to node 782 : 9
Source to node 783 : 10
Source to node 784 : 10
Source to node 785 : 9
Source to node 786 : 10
Source to node 787 : 10
Source to node 788 : 10
Source to node 789 : 9
Source to node 790 : 10
Source to node 791 : 10
Source to node 792 : 10
Source to node 793 : 10
Source to node 794 : 10
Source to node 795 : 10
Source to node 796 : 9
Source to node 797 : 10
Source to node 798 : 9
Source to node 799 : 10
Source to node 800 : 1

[0, 1]

Path from source to node 501
[0, 1]

Path from source to node 502
[0, 1]

Path from source to node 503
[0, 1, 11]

Path from source to node 504
[0, 1, 11, 112]

Path from source to node 505
[0, 1]

Path from source to node 506
[0, 1]

Path from source to node 507
[0, 1, 11, 47]

Path from source to node 508
[0, 1]

Path from source to node 509
[0, 1, 11, 30]

Path from source to node 510
[0, 1]

Path from source to node 511
[0, 1, 11]

Path from source to node 512
[0, 1, 11]

Path from source to node 513
[0, 1]

Path from source to node 514
[0, 1, 68, 117]

Path from source to node 515
[0, 1, 11, 30, 44, 48]

Path from source to node 516
[0, 1, 11, 48, 134]

Path from source to node 517
[0, 1, 11, 47]

Path from source to node 518
[0, 1, 11, 85]

Path from source to node 519
[0, 1, 11, 30, 63, 72, 87]

Path from source to node 520
[0, 1, 44]

Path from source to node 521
[0, 1, 44]

Path from source to node 522
[0, 1, 63, 68]

Path from source to node 523
[0, 1, 11, 47]

Path f

Comparing the results of the 2 variations to see if they match

In [35]:
Error = False

for i in range(len(resultA)):
    if resultA[i] == resultB[i]:
        continue
    else:
        Error = True
        break
        
if (Error):
    print("Something went wrong")
    for i in range(len(resultA)):
        print("Node", i, resultA[i] == resultB[i])
else:
    print("Output for both variations are the same")

Output for both variations are the same
